In [611]:
import pandas as pd
import datetime
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [363]:
# Limit all floats to 2 decimals
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [364]:
#load pandas DataFrame from .xlsx file, keep original data in df_raw
#With working df, drop redundant or meaningless columns
df_raw = pd.read_excel("VW_SALES_DIST_DETAILS.xlsx", sheet_name="Case Studu Extract - VW_SALES_D")
df = df_raw.drop(["Business Line Code", "Business Line Description", "Ult Dest Country", "FY 2018 Rev"], axis = 1)

In [365]:
#change data  types for some columns where data type converted to "object" by default
df = df.astype({'Fiscal Month Name': 'string', 'Item Category': 'string', 'Ship To Customer Name': 'string', \
                'Item Name': 'string', 'Item #': 'string', 'Fiscal Year':'string'})

In [566]:
# Let's take a look
df.head()

,Discount,Fiscal Date,Fiscal Month Name,Fiscal Period,Fiscal Year,Item Category,Item Name,Item #,Ship To Customer Name,Units,Revenue
0,120.75,2017-12-12,DECEMBER,6,2018,IMPURITIES,Azithromycin Related Compound F (3'-N-demethy...,1046045,A.O. Smith Corp,1,684.25
1,603.75,2017-12-01,DECEMBER,6,2018,IMPURITIES,"Anagrelide Related Compound C (Ethyl 2-(5,6-d...",1034771,A.O. Smith Corp,5,3421.25
2,34.50,2017-11-28,NOVEMBER,5,2018,SMALL MOLECULES,Amlodipine Besylate,1029501,A.O. Smith Corp,1,195.50
3,0.00,2017-11-29,NOVEMBER,5,2018,EXCIPIENTS,Succinic Acid,1623411,Abbott Laboratories,1,265.00
4,109.50,2017-12-12,DECEMBER,6,2018,IMPURITIES,"Allopurinol Related Compound C (5-(4H-1,2,4-t...",1013035,A.O. Smith Corp,1,620.50


In [567]:
# General info on loaded data: 117k rows and just 1 NULL value!!! Also, doublecheck the data types are no surprize
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117825 entries, 0 to 117824
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Discount               117825 non-null  float64       
 1   Fiscal Date            117825 non-null  datetime64[ns]
 2   Fiscal Month Name      117825 non-null  string        
 3   Fiscal Period          117825 non-null  int64         
 4   Fiscal Year            117825 non-null  string        
 5   Item Category          117825 non-null  string        
 6   Item Name              117825 non-null  string        
 7   Item #                 117825 non-null  string        
 8   Ship To Customer Name  117824 non-null  string        
 9   Units                  117825 non-null  int64         
 10  Revenue                117825 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), string(6)
memory usage: 9.9 MB


In [568]:
# Descriptive stats of numeric values in our data
df.describe()

,Discount,Fiscal Period,Units,Revenue
count,117825.00,117825.00,117825.00,117825.00
mean,135.28,6.21,2.11,819.14
std,326.89,3.37,5.57,1742.33
min,0.00,1.00,-250.00,-52912.50
25%,32.25,3.00,1.00,204.00
50%,69.20,6.00,1.00,570.35
75%,132.90,9.00,2.00,901.00
max,20756.25,12.00,500.00,117618.75


In [569]:
# Snapshot of a time period the data recorded at
print(f"First record: {df['Fiscal Date'].min()} ;  Last record: {df['Fiscal Date'].max()}")
Date_range = [df['Fiscal Date'].min(), df['Fiscal Date'].max()]

First record: 2015-07-01 00:00:00 ;  Last record: 2018-03-30 00:00:00


The discussion in this notebook will be pivoted around four questions from Case Insstructions WORD file. And the first question is:

#### Q1. How RS sales in India have trended since fiscal year 2016, by item category?
Since India is the only country mentioned in the data, I will not mention it by name on plots. Also, there are 13 item Categories and I decided to use horizontal bar chart for data viz plotting annual sales dynamics for each category, sorted by revenue. This way it is informative and readable. Sales are presented on two separate plots: one for revenue, one for volume, measured in RS units sold.

In [570]:
# Pick columns from df: "Item Category", "Fiscal Year","Revenue", "Units","Discount"
#Group by Category and Fiscal Year
# Pivot our data by unstaking MultiIndex: now data is indexed by Categories and Annual Sales are columns
sales_by_gr = df[["Item Category", "Fiscal Year","Revenue", "Units","Discount"]].groupby(["Item Category","Fiscal Year"]).sum()
sales_by_cat = sales_by_gr.unstack(level=1)

# Create two new dataframes with Revenue and Volume Sales, sotring by one of the FY
sales_by_cat_rev = sales_by_cat["Revenue"].sort_values(by=["2018"])
sales_by_cat_vol = sales_by_cat["Units"].sort_values(by=["2018"])
sales_by_cat_rev

Fiscal Year,2016,2017,2018
Item Category,,,
MISCELLANEOUS,51924.45,42718.60,30279.50
FCC,27339.55,39960.90,36322.50
VET,38170.45,73612.50,45462.75
AUTHENTIC SUBSTANCES,155816.05,165020.90,142573.00
RESIDUAL SOLVENT,197260.45,260546.00,158435.50
DIETARY SUPPLEMENTS,566913.00,585354.25,418760.55
ANTIBIOTICS,947185.25,1160897.30,817014.25
BIOTECH,1033010.55,914343.35,944058.98
EXCIPIENTS,1213188.75,1508877.60,1249402.00


In [612]:
# Use Bokeh package for plotting
# 2 plots: one with Revenue, another one with Volume sales
p_rev = sales_by_cat_rev.plot_bokeh(
    kind='barh',
    figsize=(500, 300),
    x=sales_by_cat_rev.index,
    y=["2016", "2017", "2018"],
    xlabel='Annual Sales',
    ylabel="",
    title='RS Sales by Item Category (Revenue)',
    legend = "bottom_right",
    disable_scientific_axes="x",
    zooming=False,
    panning=False,
    show_figure=False
)

p_vol = sales_by_cat_vol.plot_bokeh(
    kind='barh',
    figsize=(500, 300),
    x=sales_by_cat_vol.index,
    y=["2016", "2017", "2018"],
    xlabel='Annual Sales',
    ylabel="",
    title='RS Sales by Item Category (Volume)',
    legend = "bottom_right",
    disable_scientific_axes="x",
    zooming=False,
    panning=False,
    show_figure=False
)
pandas_bokeh.plot_grid([[p_rev,p_vol]])

Column(id='82536', ...)

There are couple things to metion from the plots above:
1) The data set contains only 3 quarters of 2018 Fiscal Year: that is whay it looks like sales drop off in 2018, in reality they are most likely continue to grow     
2) Two categories: Impurities and Small Molecules take up the most of the sales both by revenue and volume                 
3) Sales volume for Small Molecules is rougly the same as for Impurities but revenue generation lacks significantly behind, most likely due to pricing, the disscount are around 17-20% for all the categories and actually Impurities clients receive slightly more in discounts.

In [572]:
# Percentage discount added
sales_by_gr["Discount_p"] = sales_by_gr["Discount"] / sales_by_gr["Revenue"] *100
sales_by_gr

Revenue  Units   Discount  Discount_p
Item Category        Fiscal Year                                          
ANTIBIOTICS          2016          947185.25   4353  164774.95       17.40
                     2017         1160897.30   5112  145011.95       12.49
                     2018          817014.25   3651  146501.25       17.93
AUTHENTIC SUBSTANCES 2016          155816.05    607   28532.95       18.31
                     2017          165020.90    627   21533.10       13.05
                     2018          142573.00    582   28476.00       19.97
BIOTECH              2016         1033010.55   2823  189442.90       18.34
                     2017          914343.35   2514  149778.45       16.38
                     2018          944058.98   2169  168305.75       17.83
CALIBRATORS          2016         1548607.50   7721  291793.40       18.84
                     2017         1774348.15   8065  206211.25       11.62
                     2018         1652358.75   7303  272371.25       16.48
DIETARY SUPPLEMENTS  2016          566913.00   2037  146592.40       25.86
                     2017          585354.25   2088  146374.05       25.01
                     2018          418760.55   1631  120713.25       28.83
EXCIPIENTS           2016         1213188.75   6481  203927.25       16.81
                     2017         1508877.60   7352  178244.60       11.81
                     2018         1249402.00   5720  225316.75       18.03
FCC                  2016           27339.55    145    6879.45       25.16
                     2017           39960.90    197   11476.30       28.72
                     2018           36322.50    189   19077.50       52.52
IMPURITIES           2016        17452858.25  29442 3359863.20       19.25
                     2017        19164616.03  29890 2633875.80       13.74
                     2018        14692615.00  22427 2742709.75       18.67
MISCELLANEOUS        2016           51924.45    274    8504.55       16.38
                     2017           42718.60    210    5043.40       11.81
                     2018           30279.50    119    3795.00       12.53
RESIDUAL SOLVENT     2016          197260.45   1049   36382.35       18.44
                     2017          260546.00   1270   31858.50       12.23
                     2018          158435.50    793   28773.00       18.16
SMALL MOLECULES      2016         7141889.19  27359 1199617.15       16.80
                     2017         8837300.87  30994 1076817.45       12.18
                     2018         6848586.25  24667 1180501.75       17.24
SYSTEM SUITABILITY   2016         1562053.10   2998  276729.15       17.72
                     2017         1720938.75   3118  232412.45       13.50
                     2018         1294975.50   2403  216369.50       16.71
VET                  2016           38170.45    200   10867.55       28.47
                     2017           73612.50    276   10827.50       14.71
                     2018           45462.75    213   12892.25       28.36

I have also re-grouped data by Fiscal Date and  plotted Sales quarterly results by Categories.

In [577]:
# Let's re-group data by Date
sales_by_gr_ = df[["Item Category", "Fiscal Date","Revenue", "Units","Discount"]].groupby(["Item Category","Fiscal Date"]).sum()
sales_by_cat_rev_ = sales_by_gr_.unstack(level=0).resample("Q").sum()["Revenue"]
sales_by_cat_vol_ = sales_by_gr_.unstack(level=0).resample("Q").sum()["Units"]
sales_by_cat_rev_.T

Fiscal Date,2015-09-30,2015-12-31,2016-03-31,2016-06-30,2016-09-30,2016-12-31,2017-03-31,2017-06-30,2017-09-30,2017-12-31,2018-03-31
Item Category,,,,,,,,,,,
ANTIBIOTICS,214019.45,231816.25,231931.50,269418.05,243465.55,302367.60,268544.15,346520.00,231455.50,291572.25,293986.50
AUTHENTIC SUBSTANCES,36630.15,38851.50,42662.00,37672.40,34319.70,50737.20,36739.50,43224.50,35834.50,50187.25,56551.25
BIOTECH,184028.95,354626.60,228703.10,265651.90,294117.50,203115.80,161813.05,255297.00,210547.73,400491.25,333020.00
CALIBRATORS,436623.80,421024.30,349263.80,341695.60,421558.25,468280.65,367828.75,516680.50,366375.25,720662.00,565321.50
DIETARY SUPPLEMENTS,154689.75,129016.55,103714.50,179492.20,129759.65,170970.60,141903.00,142721.00,143612.75,107603.30,167544.50
EXCIPIENTS,254040.05,323077.60,273512.60,362558.50,296496.90,384314.60,374674.60,453391.50,345862.25,443583.75,459956.00
FCC,6030.75,3665.75,10661.15,6981.90,5871.90,6105.00,18444.00,9540.00,12791.25,12911.25,10620.00
IMPURITIES,4275718.50,4511249.20,3766651.75,4899238.80,4325518.60,5203087.91,4386862.30,5249147.22,4400383.00,5298351.00,4993881.00
MISCELLANEOUS,4697.75,5729.75,15633.95,25863.00,7503.60,11632.80,10081.20,13501.00,5612.00,15962.00,8705.50


In [613]:
# Use Bokeh package for plotting
# 2 line plots: one with Revenue, another one with Volume sales
p_rev_ = sales_by_cat_rev_.plot_bokeh(
    kind='line',
    figsize=(500, 300),
    ylabel="Revenue",
    title='RS Sales by Item Category (Revenue)',
    legend = False,
    disable_scientific_axes="y",
    zooming=False,
    panning=False,
    show_figure=False
)
p_vol_ = sales_by_cat_vol_.plot_bokeh(
    kind='line',
    figsize=(500, 300),
    ylabel="Sales Volume, units",
    title='RS Sales by Item Category (Volume)',
    legend = False,
    disable_scientific_axes="y",
    zooming=False,
    panning=False,
    show_figure=False
)
pandas_bokeh.plot_grid([[p_rev_,p_vol_]])

Column(id='83846', ...)

With quarterly data I put no labels to avoid plot congestion though the top 2 lines are known: Impurities and Small Molecules. The bottom ones are hardly discerned. We can see that Impurities Sales have some seasonality, and general trend is growth in all the categories. Below I plot the same data without top 2 plots to look into detailes for other categories.

In [589]:
# Same plots without Impurities and Small Molecules
# 2 line plots: one with Revenue, another one with Volume sales
sales_by_cat_rev_no_leader = sales_by_cat_rev_.drop(["IMPURITIES","SMALL MOLECULES"],axis=1)
sales_by_cat_vol_no_leader = sales_by_cat_rev_.drop(["IMPURITIES","SMALL MOLECULES"],axis=1)
p_rev_no_leaders = sales_by_cat_rev_no_leader.plot_bokeh(
    kind='line',
    figsize=(500, 300),
    ylabel="Revenue",
    title='RS Sales by Item Category (Revenue)',
    legend = False,
    disable_scientific_axes="y",
    zooming=False,
    panning=False,
    show_figure=False
)
p_vol_no_leaders = sales_by_cat_vol_no_leader.plot_bokeh(
    kind='line',
    figsize=(500, 300),
    ylabel="Sales Volume, units",
    title='RS Sales by Item Category (Volume)',
    legend = False,
    disable_scientific_axes="y",
    zooming=False,
    panning=False,
    show_figure=False
)
pandas_bokeh.plot_grid([[p_rev_no_leaders,p_vol_no_leaders]])

Column(id='70152', ...)

The only purpose of these plots is demonstrate similar type of dynamics for the rest of the Categories: some seasonality and growth. 

#### Q.2 What the item category revenue looks like as a percent of total, by fiscal year
The answer to this question based on the previous one - just need to normalize sales with aggrregate total for each Fiscal Year.

In [590]:
#Turn sales_by_cat_rev dataframe from Q1 into percentage of total form 
sales_by_cat_rev_perc = sales_by_cat_rev/sales_by_cat_rev.sum()*100
sales_by_cat_vol_perc = sales_by_cat_vol/sales_by_cat_vol.sum()*100

In [591]:
sales_by_cat_vol_perc

Fiscal Year,2016,2017,2018
Item Category,,,
MISCELLANEOUS,0.32,0.23,0.17
FCC,0.17,0.21,0.26
VET,0.23,0.30,0.30
AUTHENTIC SUBSTANCES,0.71,0.68,0.81
RESIDUAL SOLVENT,1.23,1.38,1.10
DIETARY SUPPLEMENTS,2.38,2.28,2.27
BIOTECH,3.30,2.74,3.02
SYSTEM SUITABILITY,3.51,3.40,3.34
ANTIBIOTICS,5.09,5.57,5.08


In [592]:
# Use Bokeh package for plotting
# 2 plots: one with Revenue, another one with Volume sales
p1_rev = sales_by_cat_rev_perc.plot_bokeh(
    kind='barh',
    figsize=(400,500),
    x=sales_by_cat_rev_perc.index,
    y=["2016", "2017", "2018"],
    xlabel='Annual Sales (Revenue), % of Total',
    ylabel='',
    title='RS Sales by Item Category, % of Total(Revenue)',
    legend = "bottom_right",
    zooming=False,
    panning=False, 
    show_figure=False
)

p1_vol = sales_by_cat_vol_perc.plot_bokeh(
    kind='barh',
    figsize=(400,500),
    x=sales_by_cat_vol_perc.index,
    y=["2016", "2017", "2018"],
    xlabel='Sales Volume, % of Total',
    ylabel='',
    title='RS Sales by Item Category, % of Total(Volume)',
    legend = "bottom_right",
    zooming=False,
    panning=False,
    show_figure=False
)

pandas_bokeh.plot_grid([[p1_rev,p1_vol]])

Column(id='72550', ...)

Couple comments on the plots above:
1) Impurities and Small Molecules account for  ~ 75% of total salesin revenue terms and almost 70% by volume, though their dynamics is slightly different: Impurities are losing their share of Total Revenue whereas Small Molecules gaining both in revenue and by volume            
2) In volume terms, Small Molecules took a lead in 2017 but account for about a half of Impurities' share in revenue terms most likely due to pricing.
3) Calibrators Category grew the most  in 2018 given its relatively small base 
4) Last 3-4 categoriees are hard to discern with given scale, sepatrate plots needed if there is an interest in their dynamics.

#### Q3. Sales for the top 10 items and the percent of total sales they represent


In [593]:
# Pick columns from df: "Item #" - Item names are too long, so using Ids instead, "Revenue", "Units".
# Group by Item # .
# As before, working separately with Revenue and Volume, sort both in descending order
sales_sorted_rev = df[["Item Name","Revenue", "Units"]].groupby(["Item Name"]).sum().sort_values(by=["Revenue","Units"], ascending = False)
sales_sorted_vol = df[["Item Name","Revenue", "Units"]].groupby(["Item Name"]).sum().sort_values(by=["Units"], ascending = False)

# Apply transformation to percentages, creating new columns
sales_sorted_rev = percentages_new_col(sales_sorted_rev)
sales_sorted_vol = percentages_new_col(sales_sorted_vol)

sales_sorted_rev = pd.merge(sales_sorted_rev, df[["Item #","Item Name"]], on='Item Name', how='inner').drop_duplicates().reset_index(drop=True)
sales_sorted_vol = pd.merge(sales_sorted_vol, df[["Item #","Item Name"]], on='Item Name', how='inner').drop_duplicates().reset_index(drop=True)


#Pick top-10 items by Revenue and by Sales Volume, reindex and shift indices by one to begin with 1
sales_top10_rev = sales_sorted_rev.head(10).reindex(columns= ['Item Name', 'Item #', 'Revenue','Units', 'Share_of_total_Revenue', 'Share_of_total_Units'])
sales_top10_vol = sales_sorted_vol.head(10).reindex(columns= ['Item Name', 'Item #', 'Revenue','Units', 'Share_of_total_Revenue', 'Share_of_total_Units'])



sales_top10_rev.index += 1
sales_top10_vol.index += 1

sales_top10_rev

,Item Name,Item #,Revenue,Units,Share_of_total_Revenue,Share_of_total_Units
1,Prednisone Tablets,1559505,3959177.15,17115,4.10,6.87
2,Clopidogrel Bisulfate,1140430,516109.10,594,0.53,0.24
3,Octreotide Acetate (COLD SHIPMENT REQUIRED),1477604,500270.50,813,0.52,0.33
4,Ciprofloxacin Ethylenediamine Analog (1-Cyclo...,1134324,478116.10,777,0.50,0.31
5,Prednisone,1559006,462390.80,2355,0.48,0.95
6,Sucrose,1623637,449625.90,2215,0.47,0.89
7,Gabapentin Related Compound D ((1-(3-oxo-2-az...,1287358,442627.80,696,0.46,0.28
8,Valsartan Related Compound B (N-butyryl-N-{[2...,1708784,437160.60,537,0.45,0.22
9,Clopidogrel Related Compound C (methyl (-)-(R...,1140600,428609.50,573,0.44,0.23
10,Clopidogrel Related Compound B (Methyl(+/-)-(...,1140597,406203.25,549,0.42,0.22


In [594]:
p2 = sales_top10_rev.plot_bokeh(
    kind='bar',
    figsize=(800,500),
    x="Item #",
    y=['Share_of_total_Revenue', 'Share_of_total_Units'],
    xlabel='Item ID',
    ylabel='% of Annual Sales',
    #extra_y_ranges ={"foo": Range1d(start=0, end=17000)},
    title='RS Sales by Item Category as share of Total Sales',
    legend = "top_right",
    disable_scientific_axes="y",
    zooming=False,
    panning=False,
    #show_figure=False
)
#p2.extra_y_ranges ={"foo": Range1d(start=0, end=17000)}
#pandas_bokeh.plot_grid([[p2,p1]])

Couple thing to comment on:
1) The leader of the list - Prednisone Tablets (7% by volume and 4% by revenue)     
2) No other RS reached sales of over 1% of total

#### Q.4 Is there a relationship between RS sales (Small Molecules only) and drug applications in the US?
In order to gauge whether there is a relationship  between RS sales (Small Molecules) and drug applications in the US I decided to build a  scatter polt: each dot representing a generic drug( value in Ingredient column of product.txt). Each "ingredient" can have many different trade names depending on a drug manufacturer.  Y-axis - number of applications, X-axis - sales by revenue or volume from RS items that match the ingredient composition of a drug. Since RS sales data starts at Jul 1, 2015 and average approval time for generics by FDA is  ~ 41 months or 3.5 year, I set the start-threshold for  counting approvals at Jan 1, 2019.                                                                                                                 
Simple string match was used when comparing Ingredients and Item Names which worked pretty well for "Small Molecules" fairly simple names. Most likely some matches were missed but I don't think they woukld considerably change final conclusion. However, more advance techniques could be used, such as regular expressions and "fuzzy" match based on Levenshtein Distance.
   

In [595]:
# Loading txt file from FDA Orange Book
df_apps = pd.read_csv("products.txt", sep="~")
df_apps[['DF','Route']] = df_apps["DF;Route"].str.split(";",expand=True)#
df_apps = df_apps.drop(["DF;Route"], axis = 1)
df_apps = df_apps.convert_dtypes()

# Let's clean up "Approval_Date" column before formatting it as datetime, change "Approved Prior to Jan 1, 1982" to "Jan 1, 1982"
# 
df_apps["Approval_Date"] = df_apps["Approval_Date"].str.replace("Approved Prior to Jan 1, 1982","Jan 1, 1982")
df_apps['Approval_Date'] = pd.to_datetime(df_apps['Approval_Date'], format = "%b %d, %Y")
# Only looking at RS == "Yes"
#df_apps = df_apps[df_apps["RS"]=="Yes"]

df_apps.head()

,Ingredient,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name,DF,Route
0,BUDESONIDE,UCERIS,SALIX,2MG/ACTUATION,N,205613,1,<NA>,2014-10-07,Yes,Yes,RX,SALIX PHARMACEUTICALS INC,"AEROSOL, FOAM",RECTAL
1,MINOCYCLINE HYDROCHLORIDE,AMZEEQ,FOAMIX,EQ 4% BASE,N,212379,1,<NA>,2019-10-18,Yes,Yes,RX,FOAMIX PHARMACEUTICALS INC,"AEROSOL, FOAM",TOPICAL
2,AZELAIC ACID,AZELAIC ACID,TEVA PHARMS USA,15%,A,210928,1,<NA>,2020-10-07,No,No,DISCN,TEVA PHARMACEUTICALS USA INC,"AEROSOL, FOAM",TOPICAL
3,BETAMETHASONE VALERATE,BETAMETHASONE VALERATE,PERRIGO UK FINCO,0.12%,A,78337,1,AB,2012-11-26,No,No,RX,PERRIGO UK FINCO LTD PARTNERSHIP,"AEROSOL, FOAM",TOPICAL
4,BETAMETHASONE VALERATE,BETAMETHASONE VALERATE,RICONPHARMA LLC,0.12%,A,207144,1,AB,2017-05-24,No,No,RX,RICONPHARMA LLC,"AEROSOL, FOAM",TOPICAL


In [596]:
# General info on loaded data: 39k rows with only "TE_Code" column containing NULLs, bc TE Codes assigned only to generics
df_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39433 entries, 0 to 39432
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Ingredient           39433 non-null  string        
 1   Trade_Name           39433 non-null  string        
 2   Applicant            39433 non-null  string        
 3   Strength             39377 non-null  string        
 4   Appl_Type            39433 non-null  string        
 5   Appl_No              39433 non-null  Int64         
 6   Product_No           39433 non-null  Int64         
 7   TE_Code              17819 non-null  string        
 8   Approval_Date        39433 non-null  datetime64[ns]
 9   RLD                  39433 non-null  string        
 10  RS                   39433 non-null  string        
 11  Type                 39433 non-null  string        
 12  Applicant_Full_Name  39433 non-null  string        
 13  DF                   39433 non-

In [597]:
# Couple simple helper function to process strings

# create list of lists from string array, splitting on "; ", remove extra spaces
def simple_str_processor(array):
    list_array = list(array)
    for i in range(len(list_array)):
        list_array[i] = list_array[i].strip().split("; ")
    return list_array

# create list from string array, remove extra spaces, converting tom upper case
def simple_str_processor_1(array):
    for i in range(len(array)):
        array[i] = array[i].strip().upper()
    return list(array)

# string list flattener with string concatenation, works for 2 different flagged cases with different separators 
def flatten_list(list_of_lists, flag = 'ingredients'):
    if flag == 'ingredients':
        flat_list = ['; '.join(sublist) for sublist in list_of_lists]
    elif flag == 'items':
        flat_list = [', '.join(sublist) for sublist in list_of_lists]
    else:
        flat_list = []
    return flat_list

# This func compares string array of ingridients of a given drug and  flat list of item names, matching elements added
# to a list of lists
def simple_comparison(array, list_):
    matches = [[ingridient for ingridient in el if ingridient in list_] for el in array]
    return matches    

# This function converts all the ingridient-items for a given drug into corresponding 
def items_to_sales(df_sales,df_drugs):
    Revenue, Volume = [], []
    for el in df_drugs["RS_Items"]:
        rev, vol = 0.0, 0
        for item in el.split(", "):
            rev += float(df_sales[df_sales['Item Name'] == item ]["Revenue"])
            vol += int(df_sales[df_sales['Item Name'] == item ]["Units"])
        Revenue.append(rev)
        Volume.append(vol)
    return Revenue, Volume       

In [598]:
# let's separate df_apps by date into to dataframes before/after Jul 1, 2015 - the date our RS data begind with
df_apps_before = df_apps[df_apps["Approval_Date"] < datetime.datetime.strptime("2019-01-01", '%Y-%m-%d') ]
df_apps_after = df_apps[df_apps["Approval_Date"] >= datetime.datetime.strptime("2019-01-01", '%Y-%m-%d') ]
df_apps_after.head()


,Ingredient,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name,DF,Route
1,MINOCYCLINE HYDROCHLORIDE,AMZEEQ,FOAMIX,EQ 4% BASE,N,212379,1,<NA>,2019-10-18,Yes,Yes,RX,FOAMIX PHARMACEUTICALS INC,"AEROSOL, FOAM",TOPICAL
2,AZELAIC ACID,AZELAIC ACID,TEVA PHARMS USA,15%,A,210928,1,<NA>,2020-10-07,No,No,DISCN,TEVA PHARMACEUTICALS USA INC,"AEROSOL, FOAM",TOPICAL
7,CLINDAMYCIN PHOSPHATE,CLINDAMYCIN PHOSPHATE,TARO PHARM INDS LTD,1%,A,210004,1,<NA>,2020-03-11,No,No,DISCN,TARO PHARMACEUTICAL INDUSTRIES LTD,"AEROSOL, FOAM",TOPICAL
8,CLOBETASOL PROPIONATE,CLOBETASOL PROPIONATE,GLENMARK PHARMS LTD,0.05%,A,210809,1,AB1,2019-02-15,No,No,RX,GLENMARK PHARMACEUTICALS LTD,"AEROSOL, FOAM",TOPICAL
9,CLOBETASOL PROPIONATE,CLOBETASOL PROPIONATE,GLENMARK PHARMS LTD,0.05%,A,211450,1,AB2,2019-09-09,No,No,RX,GLENMARK PHARMACEUTICALS LTD,"AEROSOL, FOAM",TOPICAL


In [599]:
#Let's use "Ingridients" column from product.txt, split it search for individual ingridient in RS data
# ingredient_list is a list, each element is a drug represented as a list of its ingridients
ingredient_list = simple_str_processor(df_apps["Ingredient"].unique())
ingredient_list_flat = flatten_list(ingredient_list)

In [600]:
# First let's narrow down RS data to  "SMALL MOLECULES" category
# rs_items_set is a set of rs items sold
df_small_mol = df[df["Item Category"]=="SMALL MOLECULES"]
rs_items_list = simple_str_processor_1(df_small_mol["Item Name"].unique())

In [601]:
# ingr_items_match is a list that matches Ingredients and  
ingr_items_match = flatten_list(simple_comparison(ingridient_list, rs_items_list),flag = 'items')

In [602]:
# Create a DataFrame with Ingredients and corresponding RS item names
ingr_items_df = pd.DataFrame(data = {'Ingredient': ingredient_list_flat , 'RS_Items': ingr_items_match})
rs_only_ingr_items_df = ingr_items_df[ingr_items_df["RS_Items"] != ""]

In [603]:
# Counting number of drug applications by Ingredients
number_of_apps = df_apps_after.groupby(['Ingredient']).count().reset_index()[['Ingredient','Trade_Name']]

In [604]:
# Preparing RS sale data including only SMALL MOLECULE category
#df_small_mol
sales_small_mol = df_small_mol[["Item Name","Revenue", "Units"]].groupby(["Item Name"]).sum().reset_index()
sales_small_mol['Item Name'] = sales_small_mol['Item Name'].str.strip()

In [605]:
#Finally building table with all necessary data: Ingredient, Sales Revenue from RS for ingridients
#and Number of drug applications
# Let's join ingr_items_df and df_apps_after
rs_vs_apps_table = pd.merge(rs_only_ingr_items_df, number_of_apps, on='Ingredient', how='inner')\
                     .rename(columns = {'Trade_Name':'Number_of_Applications'})
# and capitalize RS_Items lise the Item names in Original data table 
rs_vs_apps_table["RS_Items"] = rs_vs_apps_table["RS_Items"].str.split()\
                                .apply(lambda x: [el.capitalize() for el in x]).str.join(' ')
rs_vs_apps_table.sort_values(by=['Number_of_Applications'],ascending = False)


,Ingredient,RS_Items,Number_of_Applications
104,PREGABALIN,Pregabalin,133
290,TADALAFIL,Tadalafil,94
103,LEVOTHYROXINE SODIUM,Levothyroxine Sodium,46
261,SOLIFENACIN SUCCINATE,Solifenacin Succinate,37
88,DOXEPIN HYDROCHLORIDE,Doxepin Hydrochloride,35
...,...,...,...
201,PREDNISOLONE SODIUM PHOSPHATE,Prednisolone Sodium Phosphate,1
203,HYDROXYZINE HYDROCHLORIDE,Hydroxyzine Hydrochloride,1
207,CALCIUM CHLORIDE; POTASSIUM CHLORIDE; SODIUM C...,"Sodium Chloride, Sodium Lactate",1
208,NOREPINEPHRINE BITARTRATE,Norepinephrine Bitartrate,1


In [606]:
# sales & volume to build a dataframe
sales_col, volume_col = items_to_sales(sales_small_mol,rs_vs_apps_table)
sales_vol_df = pd.DataFrame(data = {'Revenue': sales_col , 'Units': volume_col})
rs_vs_apps_table_final = rs_vs_apps_table.join(sales_vol_df)

In [607]:
rs_vs_apps_table_final.sort_values(by=['Number_of_Applications'],ascending = False)

,Ingredient,RS_Items,Number_of_Applications,Revenue,Units
104,PREGABALIN,Pregabalin,133,10783.25,38
290,TADALAFIL,Tadalafil,94,63707.25,246
103,LEVOTHYROXINE SODIUM,Levothyroxine Sodium,46,31573.40,159
261,SOLIFENACIN SUCCINATE,Solifenacin Succinate,37,84034.45,142
88,DOXEPIN HYDROCHLORIDE,Doxepin Hydrochloride,35,14414.05,72
...,...,...,...,...,...
201,PREDNISOLONE SODIUM PHOSPHATE,Prednisolone Sodium Phosphate,1,5715.00,29
203,HYDROXYZINE HYDROCHLORIDE,Hydroxyzine Hydrochloride,1,5381.65,28
207,CALCIUM CHLORIDE; POTASSIUM CHLORIDE; SODIUM C...,"Sodium Chloride, Sodium Lactate",1,108535.15,533
208,NOREPINEPHRINE BITARTRATE,Norepinephrine Bitartrate,1,55954.70,350


In [610]:
# Use Bokeh package for plotting
# 2 plots: one with Revenue, another one with Volume sales
p4_rev = rs_vs_apps_table_final.plot_bokeh(
    kind='scatter',
    figsize=(500,500),
    x="Revenue",
    y="Number_of_Applications",
    xlabel='Annual RS Sales (Revenue)',
    ylabel='Number of US drug Applications',
    title='RS Sales by Item Category as share of Total Annual Sales (Revenue)',
    legend = False,
    disable_scientific_axes="x",
    zooming=False,
    panning=False, 
    show_figure=False
)
p4_vol = rs_vs_apps_table_final.plot_bokeh(
    kind='scatter',
    figsize=(500,500),
    x="Units",
    y="Number_of_Applications",
    xlabel='Annual RS Sales (Volume)',
    ylabel='Number of US drug Applications',
    title='RS Sales by Item Category as share of Total Annual Sales (Volume)',
    legend = False,
    disable_scientific_axes="x",
    zooming=False,
    panning=False, 
    show_figure=False
)
pandas_bokeh.plot_grid([[p4_rev,p4_vol]])

Column(id='80379', ...)

#### Conclusion.
No particular relationship was found from scatter plot: most of the dots(drugs) clustered around 0-20 applications and I don't believe that better string matching algorythm would change this. Things to do next:
1) Instead of using aggregate application numbers over entire period we can assess whether change of RS sales in time boosts applications                                     
2) Test a statistical hypothesis of whether number of applications went up after start of RS sales